# 朴素贝叶斯

一般在数据量足够，数据丰富度够的情况下，用朴素贝叶斯完成这个任务，准确度还是很不错的。

## 准备数据

In [16]:
import jieba
import pandas as pd

In [17]:
df_technology = pd.read_csv("./data/technology_news.csv", encoding="utf-8")
df_technology = df_technology.dropna()

In [18]:
df_car = pd.read_csv("./data/car_news.csv", encoding="utf-8")
df_car = df_car.dropna()

In [19]:
df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding="utf-8")
df_entertainment = df_entertainment.dropna()

In [20]:
df_military = pd.read_csv("./data/military_news.csv", encoding="utf-8")
df_military = df_military.dropna()

In [21]:
df_sports = pd.read_csv("./data/sports_news.csv", encoding="utf-8")
df_sports = df_sports.dropna()

In [22]:
technology = df_technology.content.values.tolist()[1000:210000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[1000:20000]
military = df_military.content.values.tolist()[1000: 20000]
sports = df_sports.content.values.tolist()[:20000]

In [23]:
technology[12]

'\u3000\u3000现在家里都拉了网线，都能无线上网，一定要帮他们先登上WiFi，另外，老人不懂得流量是什么，也不知道如何开关，控制流量，所以设置好流量上限很重要，免得不小心点开了视频或者下载，电话费就大发了。'

In [24]:
car[100]

'\u3000\u3000截至发稿时，人人车给出的处理方案仍旧是检修车辆。王先生则认为，车辆在购买时就存在问题，但交易平台并未能检测出来。因此，王先生希望对方退款。王先生称，他将找专业机构对车辆进行鉴定，并通过法律途径维护自己的权益。J256'

In [25]:
entertainment[10]

'\u3000\u3000不想拼赢但不想输给自己'

In [26]:
military[10]

'\xa0\xa0\xa0\xa0上世纪70年代，陆军某部“大功三连”因“煤油灯下学毛著”而享誉军内外。40多年来，这个优良传统一直在延续，连队成为闻名全军的思想工作模范连、基层建设模范连、科学发展模范连。'

In [27]:
sports[10]

'\u3000\u3000据统计，2016年仅在中国田径协会注册的马拉松赛事便达到了328场，继续呈现出爆发式增长的态势，2015年，这个数字还仅仅停留在134场。如果算上未在中国田协注册的纯“民间”赛事，国内全年的路跑赛事还要更多。'

## 分词中文文本处理

### 停用词

In [28]:
stopwords = pd.read_csv("data/stopwords.txt", index_col=False, quoting=3, sep="\t", names=["stopword"], encoding="utf-8")
stopwords = stopwords.stopword.values

## 去停用词

In [29]:
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x: len(x) > 1, segs)
            segs = filter(lambda x: x not in stopwords, segs)
            sentences.append((" ".join(segs), category))
        except Exception as e:
            print(line)
            continue

In [30]:
sentences = []

preprocess_text(technology, sentences, "technology")
preprocess_text(car, sentences, "car")
preprocess_text(entertainment, sentences, "entertainment")
preprocess_text(military, sentences, "military")
preprocess_text(sports, sentences, "sports")

## 生成训练集

In [31]:
import random
random.shuffle(sentences)

In [32]:
for sentence in sentences[:5]:
    print(sentence[0], sentence[1])

别开生面 科技 致力于 先进 虚拟化 技术 设备 别开生面 面向 高频 工作 场景 推出 工作 空间 意味着 公司 一机 多用 道路 踏出 一步 别开生面 科技 首席 营销 官段 培力 博士 透露 别开生面 科技 开发 名为 别开生面 独立 APP 登陆 大安 商店 用户 下载安装 享用 主题 空间 效应 工作 儿童 金融 游戏 场景 进一步 技术 别开生面 technology
报道 两国 开发 影响 太空 资产 敌方 轨道 飞行器 相撞 同归于尽 卫星 科茨 俄罗斯 中国 先进 卫星 活动 会合 逼近 作业 至少 活动 意在 测试 具备 固有 太空 功能 双重 用途 技术 比方说 卫星 检修 碎片 清除 目的 太空 机器人 技术 研究 用于 破坏 卫星 此类 使命 未来 一种 挑战 损害 美国 描述 太空 环境 破译 太空 活动 意图 提供 威胁 预警 能力 military
智能手机 同质化 不争 事实 两年 手机 厂商 尝试 非黑 大胆 配色 颜色 打响 市场份额 争夺战 颜色 大战 变得 尤为 激烈 technology
某次 演习 一发 航空 炸弹 机动 雷达 阵地 不远处 爆炸 方舱 震得 左摇右 人员伤亡 心有余悸 战士 发现 一段 十几厘米 弹片 穿透 铁皮 舱壁 钢板 中断 命令 并未 下达 演习 稳住 心神 雷达 官兵们 投入 military
本次 比赛 探寻出 一条 赛事 城市 合作 全新 模式 赛事 带动 整体 周边 产业 发展 赛事 辐射 方式 影响 参与 运动 项目 中来 未来 常州 WESG 系列赛 合作 支持 电子竞技 常州 推广 WESG 将会 后续 提供 赛事 相关 场馆 服务 增值 赛事 热度 留下 多余 赛事 产业 发动机 technology


## 生成测试集

In [33]:
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1111)

## 抽取词袋模型特征

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(analyzer="word", max_features=4000)
vec.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=4000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

## 训练朴素贝叶斯分类器

In [36]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## 准确率

In [37]:
classifier.score(vec.transform(x_test), y_test)

0.8350722311396469

## 抽取2-gram和3-gram的统计特征，再把词库的量放大一点

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(analyzer="word", ngram_range=(1, 4), max_features=20000)
vec.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 4), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

## 分类训练

In [39]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)
classifier.score(vec.transform(x_test), y_test)

0.8785892634207241

## 交叉验证

In [40]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score
import numpy as np

def stratifiedkfold_cv(x, y, clf_class, shuffle=True, n_folds=5, **kwargs):
    stratified_fold = StratifiedKFold(y, n_folds=n_folds, shuffle=shuffle)
    y_pred = y[:]
    for train_index, test_index in stratified_fold:
        X_train, X_test = x[train_index], x[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train, y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [41]:
NB = MultinomialNB
print(precision_score(y, stratifiedkfold_cv(vec.transform(x), np.array(y), NB), average="macro"))

0.8785842510446468


# 自己来完成一个文本分类器class

## 多种类贝叶斯

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

class TextClassifier():
    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer="word", ngram_range=(1, 4), max_features=20000)
        
    def features(self, X):
        return self.vectorizer.transform(X)
    
    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)
        
    def predict(self, x):
        return self.classifier.predict(self.features([x]))
    
    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [43]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict("这 是 有史以来 最 大 的 一 次 军舰 演习"))
print(text_classifier.score(x_test, y_test))

['military']
0.8785892634207241


## 支持向量机

In [ ]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

In [ ]:
svm = SVC(kernel="linear")
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

class TextClassifier():
    def __init__(self, classifier=SVC(kernel="linear")):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 3), max_features=120000)
        
    def featurs(self, X):
        return self.vectorizer.transform(X)
    
    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.featurs(X), y)
        
    def predict(self, x):
        return self.classifier.predict(self.featurs([x]))
    
    def score(self, X, y):
        return self.classifier.score(self.featurs(X), y)

In [ ]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict("这 是 有史以来 最 大 的 一 次 军舰 演习"))
print(text_classifier.score(x_test, y_test))